In [1]:
#Import SNN model's parameters and DIGITS dataset.
#Requires installation of Superneuromat. 
# Use pip install superneuromat.

import superneuromat as snm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import queue
import json 

from sklearn.datasets import load_digits
import pickle

from sklearn.model_selection import train_test_split

# NOTE: Python script to read in the trained model from pickle file and the spike trains 
# Simualte the network with spike trains and measure accuracy:

np.random.seed(42)

digits = load_digits()

data = digits.data
digit_labels = digits.target
labels = np.unique(digits.target)
n_labels = len(labels)

n_inputs = digits.data.shape[-1]

print('Range of pixels values:',np.max(data))

X_train, X_test, y_train, y_test = train_test_split(data, digit_labels, test_size=0.33, random_state=42)
print('Training samples:', X_train.shape, len(y_train), ' Test samples:', X_test.shape, len(y_test))


# read the pickle file:
filename = "digits_network.pkl" 
with open(filename, "rb") as file:
    infer_model = pickle.load(file)
    digits_test_spike_trains = pickle.load(file)

print(f"loaded spike train:{len(digits_test_spike_trains)}, {digits_test_spike_trains[0].shape}")


#Apply the input to the infer_model:
print('Testing the trained network on test data')

# Apply the data:
sample_count = 0           #sample count parsed so far
accuracy = 0

for sample_spikes in digits_test_spike_trains:    # across all test samples

    count = 0
    infer_model.setup()
    infer_model.input_spikes = {}
    infer_model.spike_train = []

    timestep = 0
    for all_spikes in sample_spikes:
        for n, v in enumerate(all_spikes):
            infer_model.add_spike(timestep, n+n_labels, int(v))
        timestep+=1

    infer_model.simulate(32)
    #######################################
    
    test_output = infer_model.spike_train
    test_output = np.array(test_output)
    input_spike_count = np.sum(test_output[:,10:],0)
    input_spike_count = np.reshape(input_spike_count,(8,8))

    input_spikes = test_output[:,10:]
    input_spikes_t =input_spikes.transpose()

    # check if the lable neuron has the highest spikes:
    spike_sum = np.sum(test_output[:,:10],0)
    target_spikes_observed = spike_sum[y_test[sample_count]]
    max_spikes_id = np.argmax(spike_sum)
    if max_spikes_id == y_test[sample_count]:
        accuracy+=1
        
    sample_count +=1

print('Inference metrics on test set:',accuracy, sample_count, accuracy/sample_count)
print(' n samples:', len(y_test))



Range of pixels values: 16.0
Training samples: (1203, 64) 1203  Test samples: (594, 64) 594
loaded spike train:594, (32, 64)
Testing the trained network on test data
Inference metrics on test set: 392 594 0.6599326599326599
 n samples: 594


In [2]:
import serial
import time
import matplotlib.pyplot as plt
from collections import deque
import threading
import queue
import numpy as np

# === CONFIGURATION ===
SERIAL_PORT = 'COM4'       # Replace with your actual port
BAUD_RATE = 1000000 #1000000
N = 96#20      
N_actual=N-6

# === FLAGS ===
flag_AA_en = 1
flag_FF_en = 1
flag_Thr_en = 1
flag_NIST_en = 1
flag_READ_en = 1
flag_DEBUG_en=1
flag_STDP_en=1

flag_Recieving_SPIKES=1

flag_DUMMY_DATA=0
flag_DIGITS_DATA=1
flag_plot_raster=0

num_raster=11 #Number of neurons to plot on the raster plot out of N. DIGITS needs 10


In [5]:
#Weight_AA
scaling_factor_FPGA=1

# if flag_DIGITS_DATA: #No all to all connected weights
#     weights_AA=np.zeros((N_actual,N_actual))#infer_model._weights
#     # weights_AA=weights_AA.T
#     print("The shape of weight matrix is:", weights_AA.shape)
#     print("For unscaled weights, Max weight:", np.max(weights_AA), "and Min weight:",np.min(weights_AA) )

if flag_DUMMY_DATA or flag_DIGITS_DATA: #Program as per need
    start_row = 6
    start_col = 5
    diag_size = N_actual  # 14x14 diagonal submatrix
    
    # # Precompute diagonal indices in flattened (row-major) space
    # diag_indices_AA = [(start_row + i) * N + (start_col + i) for i in range(diag_size)]
    
    # print(diag_indices_AA)
    y=0
    weights_AA = np.ones((N_actual, N_actual))*y
    # Set the value
    x = 0  # (or whatever value you want)    
    
    for start_col in range(0, N_actual, N_actual):
        end_col = start_col + N_actual
        if end_col <= N_actual:
            for i in range(N_actual):
                weights_AA[i, start_col + i] = x

# Scale first
weights_AA_scaled = weights_AA*scaling_factor_FPGA
#Clip to int8 range
weights_AA_scaled = np.clip(weights_AA_scaled, -128, 127)
padded_matrix_AA = np.zeros((N, N), dtype=np.int8)
padded_matrix_AA[6:, 5:-1] = weights_AA_scaled

# print(weights_AA_scaled)

# print("The shape of the padded weight matrix is:",padded_matrix_AA.shape)
# print(padded_matrix_AA[:,11])

flat_weights_AA=padded_matrix_AA.flatten()

# print("Datatype of weights are", flat_weights_AA.dtype)
# print("Max weight:", np.max(flat_weights_AA), "and Min weight:",np.min(flat_weights_AA) )
# print(np.where(flat_weights_AA!=0))
# print(weights_AA_scaled.shape)
# print(weights_AA_scaled[:,0:5])
# print(padded_matrix_AA[:,4])

In [8]:
#Set up Inputs_Weights refered to as W_FF
start_row_FPGA = 6
start_col_FPGA = 2

if flag_DIGITS_DATA:
    weights_FF=infer_model._weights[-64:, :10]
    # print(weights_FF)
    weights_FF=weights_FF.T
    # print(weights_FF.shape)
    # print(np.max(weights_FF))

if flag_DUMMY_DATA:
    start_row = 6
    start_col = 2
    diag_size = N_actual  # 90x90 diagonal submatrix
        # # Precompute diagonal indices in flattened (row-major) space
    diag_indices_FF = [(start_row + i) * N + (start_col + i) for i in range(diag_size)]
    weights_FF = np.zeros((10, 64))
    x = 110  # (or whatever value you want)    
    
    for start_col in range(0, 64, 10):
        end_col = start_col + 10
        if end_col <= 64:
            for i in range(10):
                weights_FF[i, start_col + i] = x
    # print("The shape of weight matrix FF is:", weights_FF.shape)

# Scale first
weights_FF_scaled = weights_FF
#Clip to int8 range
weights_FF_scaled = np.clip(weights_FF_scaled, -128, 127)
padded_matrix_FF = np.zeros((N, N), dtype=np.int8)
padded_matrix_FF[6:-80, 2:-30] = weights_FF_scaled



# print("The shape of the padded weight matrix FF is:",padded_matrix_FF.shape)
# print(padded_matrix_FF[:,11])

flat_weights_FF=padded_matrix_FF.flatten()
# flat_weights_FF=flat_weights_FF*10#[elem*10 for elem in flat_weights_FF]

# print("Datatype of weights FF are", flat_weights_FF.dtype)
# print("Max weight:", np.max(flat_weights_FF), "and Min weight:",np.min(flat_weights_FF) )
diag_check=np.where(flat_weights_FF!=0)
# print(diag_check)#tuple
# print(diag_indices_FF)#list
# if diag_check==diag_indices_FF:
#     print("Only diagonal elements have weights")
# print(weights_FF_scaled.shape)
# print(weights_FF_scaled[:,0:5])
# print(padded_matrix_FF[:,4])

In [9]:
#Read Neuron Parameters for the neurons on the FPGA
Leak = infer_model._neuron_leaks [0:10]
Ref_period=infer_model._neuron_refractory_periods_original[0:10]
Threshold= infer_model._neuron_thresholds[0:10]
Threshold_scaled=Threshold*scaling_factor_FPGA
# print("The datatype of the read parameters are:", Threshold.dtype)
# Leak=np.zeros(10)
Leak = np.concatenate((np.zeros(4, dtype=Leak.dtype), Leak, np.zeros(82, dtype=Leak.dtype)))
Ref_period = np.concatenate((np.zeros(4, dtype=Ref_period.dtype), Ref_period, np.zeros(82, dtype=Ref_period.dtype)))
Threshold_scaled = np.concatenate((np.ones(4, dtype=Threshold.dtype)*120, Threshold_scaled, np.ones(82, dtype=Threshold.dtype)*120))
# print("The Ref. period of", len(Ref_period), "neurons are:", Ref_period)
# print("The Leak of", len(Leak), "neurons are:", Leak)
# print("The Threshold of", len(Threshold_scaled), "neurons are:", Threshold_scaled)



In [10]:
#Only needed for Debugging
def send_debug_params(a):
    if flag_DEBUG_en:
        serialPort.write(b'\x40')
        print("Debug params")
        time.sleep(0.01)  
        serialPort.write(b'\x00')  # I_BIAS
        time.sleep(0.001)
        serialPort.write(b'\x00')  # 
        time.sleep(0.001)
        serialPort.write(b'\x00')  # 
        time.sleep(0.001)
        
        serialPort.write(bytes([int(a)]))# N_debug_int
        # serialPort.write(b'\x01')  # N_debug_int
        time.sleep(0.001)
        serialPort.write(b'\x00')  # 
        time.sleep(0.001)
        serialPort.write(b'\x00')  # 
        time.sleep(0.001)
        print("sent n-debug#", a)

In [11]:
#Function to recieve spikes data from the FPGA
def recieve_data_from_FPGA(result_queue):
    global receiver_stop_flag
    if flag_Recieving_SPIKES==1:
        bit_stream = []
        bytess = []
        # print("Start recieving")
        # Step 1: Wait for start marker 0xFF
        start_marker_detected = False
        while not start_marker_detected:
            if serialPort.in_waiting:
                byte = serialPort.read()
                if byte[0] == 0xFF:
                    start_marker_detected = True
                    print("Start marker detected!")
        
        # while receiver_stop_flag == 0:
        # while len(bit_stream)<=2880:
        # expected_bytes = 360
        # while len(bytess) < expected_bytes:
        while receiver_stop_flag == 0:
            if serialPort.in_waiting:
                byte = serialPort.read()
                bytess.append(byte)
                # byte_val = byte[0]
                # for i in range(8):  # LSB-first
                #     bit_stream.append((byte_val >> i) & 0x01)
        # print(f"Received {len(bytess)} bytes")

        # Step 3: Unpack bytes into bits
        for byte in bytess:
            byte_val = byte[0]
            for i in range(8):  # LSB first
                bit_stream.append((byte_val >> i) & 0x01)
        
        # print(f"Total bits received: {len(bit_stream)} bits")
        serialPort.close()
        # print(len(bytess))
        
        # print("Start decoding with overlap-aware unpacking")
        
        # Reconstruct 14-bit frames accounting for overlap
        # N_actual = 14
        spike_frames = []
        bit_buffer = []
        
        for bit in bit_stream:
            bit_buffer.append(bit)
            if len(bit_buffer) >= N_actual:
                frame = bit_buffer[:N_actual]
                spike_frames.append(frame)
                bit_buffer = bit_buffer[N_actual:]  # remove used bits, keep overlap
        
        # Convert to NumPy array
        spike_array = np.array(spike_frames).T  # shape: (96, T)
        
        # print(f"Spike matrix shape: {spike_array.shape}")
        
        # Plot raster
        if flag_plot_raster:
            plt.figure(figsize=(10, 16))
            for neuron_idx in range(N_actual):
                if neuron_idx<num_raster:
                    spike_times = np.where(spike_array[neuron_idx] == 1)[0]
                    plt.vlines(spike_times, neuron_idx + 0.5, neuron_idx + 1.5)
            
            plt.xlabel('Timestep')
            plt.ylabel('Neuron Index')
            # plt.title('Spike Raster Plot (Overlap-aware)')
            # plt.yticks(range(1, N_actual + 1))
            plt.yticks(range(1, num_raster + 1))
            plt.grid(True, linestyle='--', alpha=0.3)
            
            # plt.tight_layout()
            plt.show()              
        result_queue.put(spike_array)


In [12]:
# === SEND Weights, neuron, synapse, and learning parameters ===
def send_weights_and_params():
    if flag_AA_en:
        serialPort.write(b'\x10')  # AA command
        print("Sending AA")
        time.sleep(0.01)
        weights_bytes_AA = bytes(flat_weights_AA)
        serialPort.write(weights_bytes_AA)                 
        
        serialPort.write(b'\x00')
        
    if flag_FF_en:
        serialPort.write(b'\x00')  # FF command
        print("Sending FF")     
        weights_bytes_FF = bytes(flat_weights_FF)
        serialPort.write(weights_bytes_FF) 
        
        serialPort.write(b'\x00')

    if flag_STDP_en:
        serialPort.write(b'\x60')  # FF command
        print("Sending STDP enable matrix")
        for byte in uart_packets_AA:
            serialPort.write(bytes([byte]))
        for byte in uart_packets_FF:
            serialPort.write(bytes([byte]))
    serialPort.write(b'\x00')
    
    if flag_DUMMY_DATA or flag_DIGITS_DATA:
        if flag_Thr_en:
            serialPort.write(b'\x20')  # Threshold command
            print("Sending Thr")
            time.sleep(0.001)
            for _ in range(4):
                serialPort.write(b'\xff')  # Threshold value #af
                time.sleep(0.001)
                serialPort.write(b'\xff')  # Threshold value, 06->51%
                time.sleep(0.001)
                serialPort.write(b'\x01')  # Threshold value
                time.sleep(0.001)
            for _ in range(N-6):
                serialPort.write(b'\x00')  # Threshold value #af
                time.sleep(0.001)
                serialPort.write(b'\x90')  # Threshold value
                time.sleep(0.001)
                serialPort.write(b'\x01')  # Threshold value
                time.sleep(0.001)
            for _ in range(2):
                serialPort.write(b'\xff')  # Threshold value #af
                time.sleep(0.001)
                serialPort.write(b'\xff')  # Threshold value
                time.sleep(0.001)
                serialPort.write(b'\x01')  # Threshold value
                time.sleep(0.001)
                
            print("Sending Tref")
            serialPort.write(b'\x01')  # Scaling factor (b'\x1f')
            time.sleep(0.001)
            serialPort.write(b'\x00')  # Tref value
            time.sleep(0.001)
            serialPort.write(b'\x00')  # Tref value
            time.sleep(0.001)
            for _ in range(N-1):
                serialPort.write(b'\x00')  # Tref value (b'\x1f')
                time.sleep(0.001)
                serialPort.write(b'\x00')  # Tref value
                time.sleep(0.001)
                serialPort.write(b'\x00')  # Tref value
                time.sleep(0.001)
                
            print("Sending Tau_mem")
            for _ in range(N):
                # serialPort.write(bytes([int(0)]))
                serialPort.write(b'\x00')  # Tau_mem value 66
                time.sleep(0.001)
                serialPort.write(b'\x00')  # Tau_mem value #04
                time.sleep(0.001)
                serialPort.write(b'\x00')  # Tau_mem value
                time.sleep(0.001)
            
            print("Sending STDP_params")
            serialPort.write(b'\x7f')  # tpre_FF_max value
            time.sleep(0.001)
            serialPort.write(b'\x7f')  # tpost_FF_max value
            time.sleep(0.001)
            serialPort.write(b'\x00')  # 
            time.sleep(0.001)
            serialPort.write(b'\x7f')  # tpre__AA_max value
            time.sleep(0.001)
            serialPort.write(b'\x7f')  # tpost_AA_max value
            time.sleep(0.001)
            serialPort.write(b'\x7f')  # 
            time.sleep(0.001)             
            for _ in range(N-2):
                # serialPort.write(bytes([int(0)]))
                serialPort.write(b'\x01')  # delta_w 
                time.sleep(0.001)
                serialPort.write(b'\x01')  # 
                time.sleep(0.001)
                serialPort.write(b'\x00')  # 
                time.sleep(0.001)           


In [13]:
# === RUN SIMULATION ===
def run_simulation(NA, spike_array):
    global receiver_stop_flag
    if flag_DUMMY_DATA:
        if flag_NIST_en:
            # Prepare the big data buffer
            send_buffer = bytearray()
            dt=5
            for _ in range(1):
                # for n_address in range(NA,Y+1,1):
                for n_address in range(2,5,1):
                    # print(xxx,dt, type(xxx), type(dt))
                    send_buffer.append(dt)       # relative time
                    send_buffer.append(n_address)   # neuron index
                    send_buffer.append(0x00)      # padding
            # Append FF markers (stop signal)
            send_buffer.extend([0xFF, 0xFF, 0xFF])
            serialPort.write(b'\x30')  # Start NIST command
            print("Sending Spike Data")
            serialPort.write(send_buffer)            
            time.sleep(0.1)
            receiver_stop_flag=1
            # print("Done Spike Data", receiver_stop_flag)

    if flag_DIGITS_DATA:
        if flag_NIST_en:

            # print("Sending Spike Data")
            times, neurons = np.nonzero(spike_array)
            neuron_indices = (neurons.astype(np.uint8)+ 2).astype(np.uint8) #neurons.tolist()
            time_indices = times.astype(np.uint8) #times.tolist()
            relative_time_diffs = [time_indices[0]] + [t2 - t1 for t1, t2 in zip(time_indices[:-1], time_indices[1:])]
            relative_time_diffs = [0 if val == 1 else val for val in relative_time_diffs]
            # relative_time_diffs = np.array(relative_time_diffs, dtype=np.uint8)
            relative_time_diffs = np.clip(relative_time_diffs, 0, 255).astype(np.uint8)
        
            # Prepare the big data buffer
            send_buffer = bytearray()
            for dt, neuron in zip(relative_time_diffs, neuron_indices):
                send_buffer.append(dt)       # relative time
                send_buffer.append(neuron)   # neuron index
                send_buffer.append(0x00)      # padding

            # Append FF markers (stop signal)
            send_buffer.extend([0xFF, 0xFF, 0xFF])
            # Start sending data
            serialPort.write(b'\x30')  # Start NIST command
            serialPort.write(send_buffer)            
            time.sleep(1)
            # print("Sent STOP, waiting 1 sec")
            receiver_stop_flag= 1
            # print("Sent STOP flag",receiver_stop_flag)

In [14]:
#Setup bits STDP enable filter, if all zeros, STDP is disabled for all synapses
#DIGITS doesn't need STDP hence we set the flags to 0

# stdp_en_FF=np.zeros((N_actual,N_actual))
# Step 1: Extract and convert to binary (if needed)
# matrix = model._stdp_enabled_synapses  # Assuming it's a 2D NumPy array or similar
# binary_matrix = (np.array(matrix) > 0).astype(np.uint8)  # Ensure values are 0 or 1
# print(binary_matrix.shape, binary_matrix.dtype)
# stdp_en_AA=binary_matrix
# stdp_en_AA=np.zeros((N_actual,N_actual))

padded_stdp_enable_matrix_FF=np.zeros((N,N))
# padded_stdp_enable_matrix_FF[6:-80, 2:-30]=stdp_en_FF

padded_stdp_enable_matrix_AA=np.zeros((N,N))
# padded_stdp_enable_matrix_AA[6:, 5:-1]=stdp_en_AA

# print(padded_stdp_enable_matrix_AA.shape, padded_stdp_enable_matrix_AA.dtype)

# Assume model._stdp_enabled_synapses is your matrix (N x N)
synapse_matrix_FF = padded_stdp_enable_matrix_FF#model._stdp_enabled_synapses
synapse_matrix_AA = padded_stdp_enable_matrix_AA#model._stdp_enabled_synapses

# Step 1: Flatten in row-major order (default in NumPy)
flat_bits_FF = synapse_matrix_FF.flatten().astype(np.uint8)
flat_bits_AA = synapse_matrix_AA.flatten().astype(np.uint8)
# print(flat_bits_AA.dtype)
# Step 2: Pad to make total bit count divisible by 8
padding = (8 - (flat_bits_FF.size % 8)) % 8
if padding > 0:
    # print("CHECK THIS")
    flat_bits_FF = np.concatenate([flat_bits_FF, np.zeros(padding, dtype=np.uint8)])
    flat_bits_AA = np.concatenate([flat_bits_AA, np.zeros(padding, dtype=np.uint8)])

# Step 3: Reshape to (num_bytes, 8)
bit_groups_FF = flat_bits_FF.reshape(-1, 8)
bit_groups_AA = flat_bits_AA.reshape(-1, 8)

# Step 4: Pack LSB-first
bytes_list_FF = []
bytes_list_AA = []
for bits in bit_groups_FF:
    byte = 0
    for i, bit in enumerate(bits):  # LSB-first: bit[0] goes to b0
        byte |= (bit << i)
    bytes_list_FF.append(byte)

for bits in bit_groups_AA:
    byte = 0
    for i, bit in enumerate(bits):  # LSB-first: bit[0] goes to b0
        byte |= (bit << i)
    bytes_list_AA.append(byte)

# Final byte array
uart_packets_FF = np.array(bytes_list_FF, dtype=np.uint8)
uart_packets_AA = np.array(bytes_list_AA, dtype=np.uint8)

# print(uart_packets_AA)
# print(flat_bits_AA.shape, uart_packets_AA.dtype)
# print(np.where(flat_bits_AA==1))
# print(f"Total bytes to send: {len(uart_packets_AA)}")

In [18]:
spike_array_list=[]
flag_STDP_en=1
serialPort = serial.Serial(SERIAL_PORT, BAUD_RATE)
serialPort.reset_input_buffer()
serialPort.reset_output_buffer()
send_debug_params(1)
send_weights_and_params()
flag_plot_raster=0

global receiver_stop_flag

for j in range(len(digits_test_spike_trains)):
    if j<11:

        if j>0:
            serialPort = serial.Serial(SERIAL_PORT, BAUD_RATE)
        ####Input test spike trains from DIGITS##################################
        spike_array=np.array(digits_test_spike_trains[j])
        N_debug=int(j)
        NA=2
        print("Test image",j)
        # print("Testing Neuron", NA)
        result_queue = queue.Queue()
        receiver_stop_flag=0
        
        # Flush buffers before receiving
        serialPort.flushInput()
        serialPort.reset_input_buffer()
        serialPort.reset_output_buffer()
        thread1 = threading.Thread(target=recieve_data_from_FPGA, args=(result_queue,))
        thread1.start()
        # time.sleep(2)
        # print("Now starting")
        run_simulation(NA, spike_array) #First two arguments unused
        received_data = result_queue.get()
        thread1.join()
        spike_array_list.append(received_data)



sample_count = 0
accuracy = 0
error_flagged_samples = []  # To store indices where extra neuron spikes
predicted_labels_all=[]
print(len(spike_array_list))
for idx, spike_array in enumerate(spike_array_list):
    # spike_array shape: (90, N)
    
    # Calculate total spikes for label neurons (0-9)
    label_spike_counts = np.sum(spike_array[0:10], axis=1)  # Sum over time, per neuron (0-9)
    print(spike_array.shape, label_spike_counts)
    # Predict based on maximum spiking neuron
    predicted_label = np.argmax(label_spike_counts)
    predicted_labels_all.append(predicted_label)
    
    # Check accuracy
    if predicted_label == y_test[idx]:
        accuracy += 1
    
    # Check if any of neurons 10 to 95 fired
    if np.any(spike_array[10:90, :] > 0):
        print(f"Warning: Extra spikes detected in sample {idx}")
        error_flagged_samples.append(idx)

    sample_count += 1

print('Inference metrics on FPGA test set:', accuracy, sample_count, "Accuracy:", accuracy/sample_count)
print('Total samples flagged with extra spikes:', len(error_flagged_samples))
print('Indices with extra spikes:', error_flagged_samples)
        

Debug params
sent n-debug# 1
Sending AA
Sending FF
Sending STDP enable matrix
Sending Thr
Sending Tref
Sending Tau_mem
Sending STDP_params
Test image 0
Start marker detected!
Test image 1
Start marker detected!
Test image 2
Start marker detected!
Test image 3
Start marker detected!
Test image 4
Start marker detected!
Test image 5
Start marker detected!
Test image 6
Start marker detected!
Test image 7
Start marker detected!
Test image 8
Start marker detected!
Test image 9
Start marker detected!
Test image 10
Start marker detected!
11
(90, 567) [21 27 24 20 26 24 40 20 30 22]
(90, 689) [28 30 31 36 28 36 31 30 35 34]
(90, 535) [20 26 28 35 17 28 27 22 30 26]
(90, 559) [20 26 22 22 25 27 24 28 27 21]
(90, 595) [22 30 33 29 19 27 33 20 31 21]
(90, 639) [21 32 34 24 24 32 33 19 32 21]
(90, 635) [25 32 30 27 28 37 32 29 32 21]
(90, 515) [21 27 29 28 22 27 24 28 29 23]
(90, 557) [19 26 26 23 24 34 26 25 26 17]
(90, 627) [22 29 35 31 24 28 27 28 31 20]
(90, 555) [19 38 32 31 27 26 26 31 33 22]